# Scan-to-BIM Bootcamp Week 2

*This session will discuss sensor data fusion for generating point clouds, point cloud data formats, point cloud visualization tools such as CloudCompare, plotly, and point cloud registration algorithms.*

The assignment for this notebook is to perform point cloud registration using Numpy operations for three laser-scanned point clouds from a construction site.

# 1. Point Cloud Data Loading and Visualization

In [ ]:
# Install the Open3D library
# This step is needed because Open3D is not a standard library included in Google Colab
!pip install open3d

In [1]:
# Mount a Google Drive folder so that the data files can be accessed
from google.colab import drive
from google.colab import files
import sys
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/Scan-to-BIM Bootcamp/Week 2/
sys.path.insert(0,'/content/drive/MyDrive/Scan-to-BIM Bootcamp/Week 2/')

Mounted at /content/drive
/content/drive/MyDrive/Scan-to-BIM Bootcamp/Week 2


In [2]:
# Import libraries and utility functions
import open3d as o3d
import numpy as np
from utils import draw_geometries

In [4]:
# Load in the input point cloud using the Open3D library
pcd_object = o3d.io.read_point_cloud("registration_1_small.ply")
# Convert to a NumPy array for further processing
initial_pcd = np.hstack([pcd_object.points, pcd_object.colors]).astype(np.float32)
print("Point cloud dimensions are: ", initial_pcd.shape)
np.set_printoptions(precision=3, suppress=True)
print("Point cloud coordinates are:")
print(initial_pcd)

Point cloud dimensions are:  (86650, 6)
Point cloud coordinates are:
[[ -0.036  -0.046 288.149   0.451   0.294   0.263]
 [  0.046   0.021 288.147   0.475   0.275   0.208]
 [  0.129   0.09  288.146   0.435   0.255   0.161]
 ...
 [  3.495   2.93  288.387   0.788   0.667   0.659]
 [  2.491   2.086 284.53    0.392   0.196   0.153]
 [  6.992   5.79  288.485   0.996   0.996   0.996]]


In [5]:
# Visualize the point cloud using a 3D web viewer
draw_geometries([pcd_object])

Output hidden; open in https://colab.research.google.com to view.

# 2. Loading multiple point clouds

In [8]:
# Load point clouds 

pcd_object_1 = o3d.io.read_point_cloud("registration_1.ply")
pcd_npy_1 = np.hstack([pcd_object_1.points, pcd_object_1.colors]).astype(np.float32)
print("Point cloud (1) dimensions are: ", pcd_npy_1.shape)

pcd_object_2 = o3d.io.read_point_cloud("registration_2.ply")
pcd_npy_2 = np.hstack([pcd_object_2.points, pcd_object_2.colors]).astype(np.float32)
print("Point cloud (2) dimensions are: ", pcd_npy_2.shape)

pcd_object_3 = o3d.io.read_point_cloud("registration_3.ply")
pcd_npy_3 = np.hstack([pcd_object_3.points, pcd_object_3.colors]).astype(np.float32)
print("Point cloud (3) dimensions are: ", pcd_npy_3.shape)

Point cloud (1) dimensions are:  (86650, 6)
Point cloud (2) dimensions are:  (117521, 6)
Point cloud (3) dimensions are:  (113852, 6)


In [9]:
# Visualize multiple point clouds
draw_geometries([pcd_object_1, pcd_object_2, pcd_object_3])

Output hidden; open in https://colab.research.google.com to view.

# 3. Rotation and Translation

In [10]:
# Transformation parameters for aligning point cloud #2 with point cloud #1

rotation_2_to_1 = np.array([[0.432885169983, -0.901445329189, -0.002608089475], 
[0.901447117329, 0.432888448238, -0.000844127964],
[0.001889946987, -0.001985644223, 0.999996244907]])

translation_2_to_1 = np.array([-8.127865791321, 4.851447582245, -0.801156997681])


In [15]:
# Apply transformation to point cloud #2

xyz_original = pcd_npy_2[:, :3]
xyz_transformed = rotation_2_to_1.dot(xyz_original.T).T + translation_2_to_1

pcd_npy_2[:, :3] = xyz_transformed
pcd_object_2.points = o3d.utility.Vector3dVector(xyz_transformed)

In [12]:
# Transformation parameters for aligning point cloud #3 with point cloud #1

rotation_3_to_1 = np.array([[0.803440749645, -0.595375716686, -0.003266356653], 
[0.595375061035, 0.803446888924, -0.001285545644],
[0.003389726859, -0.000911847514, 0.999993860722]])

translation_3_to_1 = np.array([-10.877108573914, -5.710428237915, -0.970873713493])

In [16]:
# Apply transformation to point cloud #3

xyz_original = pcd_npy_3[:, :3]
xyz_transformed = rotation_3_to_1.dot(xyz_original.T).T + translation_3_to_1

pcd_npy_3[:, :3] = xyz_transformed
pcd_object_3.points = o3d.utility.Vector3dVector(xyz_transformed)

In [14]:
# Visualize registration results
draw_geometries([pcd_object_1, pcd_object_2, pcd_object_3])

Output hidden; open in https://colab.research.google.com to view.

# 4. Save results

In [17]:
# Use the "vstack" function to combine multiple arrays into a single array
pcd_npy_combined = np.vstack([pcd_npy_1, pcd_npy_2, pcd_npy_3])
print("Combined point cloud dimensions are: ", pcd_npy_combined.shape)

Combined point cloud dimensions are:  (318023, 6)


In [18]:
pcd_object.points = o3d.utility.Vector3dVector(pcd_npy_combined[:, 0:3])
pcd_object.colors = o3d.utility.Vector3dVector(pcd_npy_combined[:, 3:6])

o3d.io.write_point_cloud('output.ply', pcd_object)

True